# notebook in which all figures are created 

## regimes of dynamical systmes

In [1]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT

In [6]:
function lorenz_regimes()
    tss = [create_series("lorenz", 20    , Δt=0.01)]
    push!(tss, create_series("lorenz", 22, Δt=0.01))
    push!(tss, create_series("lorenz", 24, Δt=0.01))
    push!(tss, create_series("lorenz", 25, Δt=0.01))
    push!(tss, create_series("lorenz", 28, Δt=0.01))
    push!(tss, create_series("lorenz", 34, Δt=0.01))
    ps = (plot3d(tss[i][:, 1], tss[i][:, 2], tss[i][:, 3]) for i in 1:6)
    g = plot(ps..., layout=6, 
            title=["\n20" "ρ as bifurcation parameter\n22" "\n24" "\n25" "\n28" "\n34"], titlefont=font(11), 
            legend=nothing, 
            plot_title="3d lorenz regimes", plot_titlevspan=0.08)
    mkpath("../Figures/lorenz/") 
    savefig(g, "../Figures/lorenz/lorenz_regimes.png")
end

function bursting_neuron_regimes()
    tss = [create_series("bursting_neuron", 2)]
    push!(tss,  create_series("bursting_neuron", 3))
    push!(tss,  create_series("bursting_neuron", 5))
    push!(tss,  create_series("bursting_neuron", 9))
    push!(tss,  create_series("bursting_neuron", 10))
    push!(tss,  create_series("bursting_neuron", 10.2))
    ps = (plot3d(tss[i][:, 1], tss[i][:, 2], tss[i][:, 3]) for i in 1:6)
    g = plot(ps..., layout=6,
        title=["\n2" "\$g_{nmda}\$ as bifurcation parameter\n3" "\n5" "\n9" "\n10" "\n10.2"], titlefont=font(11),
        legend=nothing,
        plot_title="3d bursting neuron regimes", plot_titlevspan=0.08,
        size=(500, 500)
    )
    mkpath("../Figures/bursting_neuron/")
    savefig(g, "../Figures/bursting_neuron/bursting_neuron_regimes.png")
end

bursting_neuron_regimes (generic function with 1 method)

In [7]:
lorenz_regimes()
bursting_neuron_regimes()

UndefVarError: UndefVarError: bursting_neuron_regime not defined

## EVD analysis
### lorenz

In [1]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT
using LinearAlgebra
using Measures

In [2]:
function lorenz_evd()
    function remove!(a, item)
        deleteat!(a, findall(x -> x == item, a))
    end

    experiments = readdir("../Results")
    remove!(experiments, "default")
    deleteat!(experiments, findall(x -> occursin("bursting", x), experiments))

    # load all the experiments
    for exp in experiments
        model_path = load_result_path(exp, 5000, 1)
        m = BPTT.load_model(model_path)

        # copy all images
        dst_path = "lorenz_evd/$(exp)_traj.png"
        cp(load_result_path(exp, 5000, 1; img=true), dst_path, force=true)


        # compose single matrix AW
        AW = diagm(m.A) + m.W

        # get evd of AW
        EVD = eigen(AW)
        λs = EVD.values
        Vs = EVD.vectors
        try
            push!(exp_λs, λs)
            push!(exp_Vs, Vs)
        catch e
            global exp_λs = [λs]
            global exp_Vs = [Vs]
        end
    end

    # for making subplots for each experiment
    function make_subplots(exp, plots, type)
        if exp == "lorenz"
            μ = "ρ"
            title = ["\n20" "\n22" "      ρ as bifurcation parameter\n24" "\n25" "\n28" "\n34"]
        else
            μ = "\$g_{nmda}\$"
            title = ["\n2" "\n3" "        $μ as bifurcation parameter\n5" "\n9" "\n10" "\n10.2"]
        end
        p = plot(plots..., layout=grid(1, 6),
            title=title, titlefont=font(11),
            plot_title="$type of evd with $exp plrnn", plot_titlevspan=0.08,
            size=(1000, 500)
        )
        savefig(p, "lorenz/$(exp)_$(type)_evd.png")
    end

    # convert to array
    arr_exp_l = reduce(hcat, exp_λs)

    # get the sorting vectors
    for lambdas in exp_λs
        v = sortperm(abs.(lambdas))
        try
            push!(sortv, v)
        catch
            global sortv = [v]
        end
    end

    # Plots of single experiments

    # sort the absolute values for comparison
    abs_λ = [abs.(exp_λs[i][sortv[i]]) for i in 1:size(exp_λs)[1]]

    # histogram of all abs values
    hists_abs_val = (histogram(abs_val, bins=8, legend=nothing, xlabel="value [b.E.]", ylabel="Hits") for abs_val in abs_λ)

    # im and real part in one figure
    for (i, lambdas) in enumerate(exp_λs)
        imre = scatter(imag(lambdas[sortv[i]]), label="imag", legend=:topleft, xlabel="Ev No")
        scatter!(imre, real(lambdas[sortv[i]]), label="real", legend=:topleft, xlabel="Ev No")
        try
            push!(im_res, imre)
        catch
            global im_res = [imre]
        end
    end


    # abs values together
    c = [:red, :red, :turquoise, :green, :black, :black]
    label = ["limit cycle", "limit cycle", "limit cycle", "right after bifurcation", "chaotic regime", "chaotic regime"]
    for (i, abs_val) in enumerate(abs_λ)
        try
            plot!(abs_plot, abs_val, 1:15, c=c[i], xlabel="Ev value [b.E.]",
                ylabel="Ev No", label=label[i])
        catch
            global abs_plot = plot(abs_val, 1:15,
                c=c[i],
                label=label[i],
                xlabel="Ev value [b.E.]",
                ylabel="Ev No",
                plot_title="Eigenvalue evolution", plot_titlevspan=0.05)
        end
    end
    savefig(abs_plot, "lorenz_evd/change_in_abs_vals.png")


    make_subplots("lorenz", hists_abs_val, "hist of abs val")
    make_subplots("lorenz", im_res, "im and real")

    # look at change of the parameters
    for id in 2:6 # for all experiments
        try
            push!(change_abs, abs.(abs_λ[id] - abs_λ[id-1]))
        catch e
            global change_abs = [abs.(abs_λ[id] - abs_λ[id-1])]
        end
    end
    arr_change_abs = reduce(hcat, change_abs)

    for id in 2:6 # for all experiments
        try
            push!(change_abs_p, abs.(abs_λ[id] ./ abs_λ[id-1]))
        catch e
            global change_abs_p = [abs.(abs_λ[id] ./ abs_λ[id-1])]
        end
    end
    arr_change_abs_p = reduce(hcat, change_abs_p)

    # hist of overall change in all eigenvalues
    # hist of overall change in eigenvalues
    hist_abs = histogram(collect(Iterators.flatten(change_abs_p).-1),
        title="change of all eigenvalues",
        legend=nothing, bins=15)

    # hist of change of the values itselfs
    hist_lambda_abs = scatter(sum(arr_change_abs, dims=2), title="sum of change of eigenvalues", legend=nothing, xlabel="Ev No")
    vline!(hist_lambda_abs, [12, 11, 15], c=:blue, linewidth=1, label="detailed plots")

    savefig(hist_abs, "lorenz_evd/overall_cahnge_abs_val.png")
    savefig(hist_lambda_abs, "lorenz_evd/change_of_ev.png")

    # plot the change of some of the imoprtant eigenvalues
    val, idx = findmax(sum(arr_change_abs, dims=2))
    arr_abs_λ = reduce(hcat, abs_λ)

    # specific plot, to show different changes
    a = plot(arr_abs_λ[end-3, :], xlabel="experiment", ylabel="b.E.")
    b = plot(arr_abs_λ[end-4, :], xlabel="experiment", ylabel="b.E.")
    c = plot(arr_abs_λ[idx[1], :], xlabel="experiment", ylabel="b.E.")
    d = plot(a, b, c, layout=grid(1, 3),
        size=(1000, 500),
        legend=nothing,
        margin=10mm,
        plot_title="different changes of absolute values", plot_titlevspan=0.05)
    savefig(d, "lorenz_evd/ev_t_cross1.png")

    # heatmap for all eigenvalues
    heat = heatmap(abs.(arr_abs_λ), c=:thermal, xlabel="experiment", ylabel="eigenvalue") 
    savefig(heat, "lorenz_evd/heatmap.png")
end

lorenz_evd (generic function with 1 method)

In [3]:
lorenz_evd()


### bursting neuron

In [1]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT
using LinearAlgebra
using Measures

In [2]:
function bursting_neuron_evd()

    function remove!(a, item)
        deleteat!(a, findall(x -> x == item, a))
    end

    experiments = readdir("../Results")
    remove!(experiments, "default")
    deleteat!(experiments, findall(x -> occursin("lorenz", x), experiments))
    permute!(experiments, [2, 3, 4, 5, 1])

    # load all the experiments
    for exp in experiments
        model_path = load_result_path(exp, 5000, 1)
        m = BPTT.load_model(model_path)

        # copy all images
        dst_path = "bursting_neuron_evd/$(exp)_traj.png"
        cp(load_result_path(exp, 5000, 1; img=true), dst_path, force=true)


        # compose single matrix AW
        AW = diagm(m.A) + m.W

        # get evd of AW
        EVD = eigen(AW)
        λs = EVD.values
        Vs = EVD.vectors
        try
            push!(exp_λs, λs)
            push!(exp_Vs, Vs)
        catch e
            global exp_λs = [λs]
            global exp_Vs = [Vs]
        end
    end

    # for making subplots for each experiment
    function make_subplots(exp, plots, type)
        if exp == "lorenz"
            μ = "ρ"
            title = ["\n20" "\n22" "      ρ as bifurcation parameter\n24" "\n25" "\n28" "\n34"]
        else
            μ = "\$g_{nmda}\$"
            title = ["\n2" "\n3" "     $μ as bifurcation parameter\n5" "\n9" "\n10" "\n10.2"]
        end
        p = plot(plots..., layout=grid(1, 5),
            title=title, titlefont=font(11),
            plot_title="$type of evd with $exp plrnn", plot_titlevspan=0.08,
            size=(1000, 500)
        )
        savefig(p, "bursting_neuron_evd/$(exp)_$(type)_evd.png")
    end


    # convert to array
    arr_exp_l = reduce(hcat, exp_λs)

    # get sorting vectors
    for lambdas in exp_λs
        v = sortperm(abs.(lambdas))
        try
            push!(sortv, v)
        catch
            global sortv = [v]
        end
    end



    # Plots of single experiments

    # sort the absolute values for comparison
    abs_λ = [abs.(exp_λs[i][sortv[i]]) for i in 1:size(exp_λs)[1]]

    # histogram of all abs values
    hists_abs_val = (histogram(abs_val, bins=15, legend=nothing, xlabel="value [b.E.]", ylabel="Hits") for abs_val in abs_λ)

    # im and real part in one figure
    for (i, lambdas) in enumerate(exp_λs)
        imre = scatter(imag(lambdas[sortv[i]]), label="imag", legend=:topleft, xlabel="Ev No", ylim=(-2.5, 2.5))
        scatter!(imre, real(lambdas[sortv[i]]), label="real", legend=:topleft, xlabel="Ev No", ylim=(-2.5, 2.5))
        try
            push!(im_res, imre)
        catch
            global im_res = [imre]
        end
    end

    make_subplots("bursting_neuron", hists_abs_val, "hist of abs val")
    make_subplots("bursting_neuron", im_res, "im and real")

    # abs values together
    c = [:orange, :red, :darkred, :purple, :black]
    label = ["cycle", "1 loop", "4 loops", "almost bursting", "bursting"]
    for (i, abs_val) in enumerate(abs_λ)
        try
            plot!(abs_plot, abs_val, 1:26, c=c[i],
                xlabel="Ev value [b.E.]",
                ylabel="Ev No", label=label[i])
        catch
            global abs_plot = plot(abs_val, 1:26,
                c=c[i],
                label=label[i],
                xlabel="Ev value [b.E.]",
                ylabel="Ev No",
                plot_title="Eigenvalue evolution", plot_titlevspan=0.05)
        end
    end
    savefig(abs_plot, "bursting_neuron_evd/abs_val_evol.png")

    # look at change of the parameters
    for id in 2:5 # for all experiments
        try
            push!(change_abs, abs.(abs_λ[id] - abs_λ[id-1]))
        catch e
            global change_abs = [abs.(abs_λ[id] - abs_λ[id-1])]
        end
    end
    arr_change_abs = reduce(hcat, change_abs)

    for id in 2:5 # for all experiments
        try
            push!(change_abs_p, abs.(abs_λ[id] ./ abs_λ[id-1]))
        catch e
            global change_abs_p = [abs.(abs_λ[id] ./ abs_λ[id-1])]
        end
    end
    arr_change_abs_p = reduce(hcat, change_abs_p)


    # hist of overall change in eigenvalues
    hist_abs = histogram(collect(Iterators.flatten(change_abs_p) .- 1),
        title="change of all eigenvalues",
        legend=nothing, bins=15)

    # hist of change of the values itselfs
    hist_lambda_abs = scatter(sum(arr_change_abs, dims=2),
        title="sum of change of eigenvalues",
        legend=nothing, xlabel="Ev No")
    vline!(hist_lambda_abs, [14, 13, 26], c=:blue, linewidth=1, label="detailed plots")
        
    savefig(hist_lambda_abs, "bursting_neuron_evd/change_of_ev.png")
    savefig(hist_abs, "bursting_neuron_evd/overall_change_abs_val.png")

    # plot the change of some of the imoprtant eigenvalues
    val, idx = findmax(sum(arr_change_abs, dims=2))
    arr_abs_λ = reduce(hcat, abs_λ)

    # specific plot, to show different changes
    a = plot(arr_abs_λ[end-12, :], xlabel="experiment", ylabel="b.E.")
    b = plot(arr_abs_λ[end-13, :], xlabel="experiment", ylabel="b.E.")
    c = plot(arr_abs_λ[idx[1], :], xlabel="experiment", ylabel="b.E.")
    d = plot(a, b, c, layout=grid(1, 3),
                        size=(1000, 500),
                        legend=nothing,
                        margin=10mm,
                        plot_title="different changes of absolute values", plot_titlevspan=0.05)

    savefig(d, "bursting_neuron_evd/ev_t_cross1.png")

    # heatmap for all eigenvalues
    heat = heatmap(abs.(arr_abs_λ), c=:thermal)
    savefig(heat, "bursting_neuron_evd/heatmap.png")
end

bursting_neuron_evd (generic function with 1 method)

In [3]:
bursting_neuron_evd()

UndefVarError: UndefVarError: heat not defined

# evd of A+WD

In [1]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT
using LinearAlgebra
using Measures
using SplitApplyCombine


In [2]:
function evd_all(bins::Int64, system::String)

    bins = bins
    exp_str = system
    dir = "../Figures/$(exp_str)_all"
    function remove!(a, item)
        deleteat!(a, findall(x -> x == item, a))
    end

    experiments = readdir("../Results")
    remove!(experiments, "default")
    deleteat!(experiments, findall(x -> !occursin(system, x), experiments))

    # load all the experiments
    for exp in experiments
        model_path = load_result_path(exp, 5000, 1)
        m = BPTT.load_model(model_path)

        # copy all images
        mkpath(dir)
        dst_path = "$dir/$(exp)_traj.png"
        # cp(load_result_path(exp, 5000, 1; img=true), dst_path, force=true)

        # safe the latent dim of the plrnn
        dim = length(m.A)

        # loop over all possible matrices D
        all_Ds = ([0, 1] for i in 1:dim)
        global all_λs = nothing
        global all_λs = Vector{Vector{ComplexF32}}()
        for (i, D) in enumerate(collect(Iterators.product(all_Ds...)))
            Dt = diagm(collect(D))

            # compose single matrix A+WD
            AWD = diagm(m.A) + m.W * Dt

            # get evd of AWD
            EVD = eigen(AWD)
            λs = EVD.values
            Vs = EVD.vectors
            # add the same for Vs by doing the same with labda->v
            push!(all_λs, complex(λs))
        end
        try
            push!(exp_λs, all_λs)
        catch e
            if typeof(e) == UndefVarError
                global exp_λs = [all_λs]
            else
                rethrow(e)
            end
        end
    end

    model_path = load_result_path(experiments[1], 5000, 1)
    m = BPTT.load_model(model_path)
    dim = length(m.A)
    #just a few checks
    @assert length(exp_λs) == length(experiments)
    @assert typeof(exp_λs) == Vector{Vector{Vector{ComplexF32}}}
    @assert [length(exp_λs[i]) for i in 1:length(experiments)] == [2^length(m.A) for i in 1:length(experiments)]

    # for making subplots for each experiment
    function make_subplots(exp, plots, type)
        if exp == "lorenz"
            μ = "ρ"
            title = ["\n20" "ρ as bifurcation parameter\n22" "\n24" "\n25" "\n28" "\n34"]
        else
            μ = "\$g_{nmda}\$"
            title = ["\n2" "$μ as bifurcation parameter\n3" "\n5" "\n9" "\n10" "\n10.2"]
        end
        p = plot(plots..., layout=length(plots),
            title=title, titlefont=font(11),
            plot_title="$type of evd with \n $exp plrnn", plot_titlevspan=0.15
        )
        type_str = replace(type, " " => "_") # replace the spaces for savefig
        savefig(p, "../Figures/$(exp)_all/$(exp)_$(type_str)_evd.png")
    end

    # convert to array
    arr_l = combinedims(exp_λs)
    arr_l = combinedims(arr_l)
    exp_λs = splitdims(arr_l)
    arr_l = permutedims(arr_l, (3, 2, 1))

    # sort all the arrays
    abs_ls_sort = zeros(6, 2^dim, dim)
    res_ls_sort = zeros(6, 2^dim, dim)
    ims_ls_sort = complex(zeros(6, 2^dim, dim))

    for (ex, all_lambdas) in enumerate(exp_λs)
        abs_ls = abs.(all_lambdas)
        res_ls = real(all_lambdas)
        ims_ls = imag(all_lambdas)
        v = nothing
        for i in 1:size(all_lambdas)[2]
            v = sortperm(abs_ls[:, i])
            abs_ls_sort[ex, i, :] = abs_ls[v, i]
            res_ls_sort[ex, i, :] = res_ls[v, i]
            ims_ls_sort[ex, i, :] = ims_ls[v, i]
        end
    end

    # Plots of single experiments
    # make it a vector for easy iteration
    abs_λs = splitdims(abs_ls_sort, 1)
    ims_λs = splitdims(ims_ls_sort, 1)
    res_λs = splitdims(res_ls_sort, 1)


    # histogram of all abs values
    hists_abs_val = (histogram(collect(Iterators.flatten(abs_val)), yaxis=:log, bins=bins, legend=nothing, xlabel="Ev value [b.E.]", ylabel="Hits") for abs_val in abs_λs)

    # im and real part histograms
    hists_ims = (histogram(collect(Iterators.flatten(ims_val)), yaxis=:log, bins=bins, xlabel="Ev value [b.E.]") for ims_val in ims_λs)
    hists_res = (histogram(collect(Iterators.flatten(res_val)), yaxis=:log, bins=bins, xlabel="Ev value [b.E.]") for res_val in res_λs)

    make_subplots(exp_str, hists_abs_val, "histogram of absolute values")
    make_subplots(exp_str, hists_ims, "histogram of imaginary part")
    make_subplots(exp_str, hists_res, "historam of real parts")

    _, index = findmax(exp_λs[1])

    # abs values together for one case
    if exp_str == "lorenz"
        c = [:red, :red, :turquoise, :green, :black, :black]
        label = ["limit cycle", "limit cycle", "limit cycle", "right after bifurcation", "chaotic regime", "chaotic regime"]
    else
        c = [:orange, :red, :darkred, :purple, :black]
        label = ["cycle", "1 loop", "4 loops", "almost bursting", "bursting"]
    end
    for (i, abs_val) in enumerate(abs_λs)
        try
            plot!(abs_plot, abs_val[index[1], :], 1:dim, c=c[i], xlabel="Ev value [b.E.]",
                ylabel="Ev No", label=label[i])
        catch
            global abs_plot = plot(abs_val[index[1], :], 1:dim,
                c=c[i],
                label=label[i],
                xlabel="Ev value [b.E.]",
                ylabel="Ev No",
                plot_title="Eigenvalue evolution of quadrant $(index[1])", plot_titlevspan=0.05)
        end
    end
    savefig(abs_plot, "../Figures/lorenz_all/change_in_abs_vals.png")

    # look at change of the parameters
    for id in 2:length(experiments) # for all experiments
        try
            push!(change_abs, abs.(abs_λs[id] - abs_λs[id-1]))
        catch e
            global change_abs = [abs.(abs_λs[id] - abs_λs[id-1])]
        end
    end

    # percentual change
    for id in 2:length(experiments) # for all experiments
        try
            push!(change_abs_p, abs.(abs_λs[id] ./ abs_λs[id-1]))
        catch e
            global change_abs_p = [abs.(abs_λs[id] ./ abs_λs[id-1])]
        end
    end
    arr_change_abs = combinedims(change_abs, 1)
    arr_change_abs_p = combinedims(change_abs_p, 1)

    idcs = findall(x -> x >= 2, arr_change_abs_p)
    for id in idcs
        arr_change_abs_p[id] = 0
    end
    
    # hist of overall change in all eigenvalues
    hist_abs_p = histogram(collect(Iterators.flatten(arr_change_abs_p .- 1)),
        title="percentual change of all eigenvalues",
        yaxis=:log,
        legend=nothing, bins=bins)

    # hist of overall change in all eigenvalues absou change
    hist_abs = histogram(collect(Iterators.flatten(arr_change_abs)),
        yaxis=:log,
        title="absolute change of all eigenvalues",
        legend=nothing, bins=bins)


    savefig(hist_abs_p, "../Figures/lorenz_all/overall_cahnge_abs_val_p.png")
    savefig(hist_abs, "../Figures/lorenz_all/overall_cahnge_abs_val.png")

    # plot the change of some of the imoprtant eigenvalues
    val, idx = findmax(sum(arr_change_abs, dims=1))
    val_p, idx_p = findmax(sum(arr_change_abs_p, dims=1))

    # specific plot, to show change of the most changing evs
    a = plot(abs_ls_sort[:, idx[2], idx[3]], xlabel="experiment", ylabel="b.E.")
    b = plot(abs_ls_sort[:, idx_p[2], idx_p[3]], xlabel="experiment", ylabel="%")

    p = plot(a, b, layout=2,
        size=(1000, 500),
        legend=nothing,
        margin=10mm,
        title=["absolute change" "percentual change"],
        plot_title="detailed changes of absolute values", plot_titlevspan=0.05)
    savefig(d, "../Figures/lorenz_all/ev_t_detail.png")
end

lorenz_evd_all (generic function with 1 method)

In [3]:
lorenz_evd_all(20, "lorenz")

┌ Error: Some Julia code in the VS Code extension crashed
└ @ VSCodeServer /home/patrick/.vscode/extensions/julialang.language-julia-1.6.28/scripts/error_handler.jl:15
ERROR: 

TypeError: in isdefined, expected Symbol, got a value of type Tuple{Int64, Core.Compiler.NewSSAValue}
Stacktrace:
 [1] getvariables(show_modules::Bool)
   @ VSCodeServer ~/.vscode/extensions/julialang.language-julia-1.6.28/scripts/packages/VSCodeServer/src/trees.jl:277
 [2] #invokelatest#2
   @ ./essentials.jl:716 [inlined]
 [3] invokelatest
   @ ./essentials.jl:714 [inlined]
 [4] repl_getvariables_request(conn::VSCodeServer.JSONRPC.JSONRPCEndpoint{Base.PipeEndpoint, Base.PipeEndpoint}, params::NamedTuple{(:modules,), Tuple{Bool}})
   @ VSCodeServer ~/.vscode/extensions/julialang.language-julia-1.6.28/scripts/packages/VSCodeServer/src/trees.jl:269
 [5] dispatch_msg(x::VSCodeServer.JSONRPC.JSONRPCEndpoint{Base.PipeEndpoint, Base.PipeEndpoint}, dispatcher::VSCodeServer.JSONRPC.MsgDispatcher, msg::Dict{String, Any})
   @ VSCodeServer.JSONRPC ~/.vscode/extensions/julialang.language-julia-1.6.28/scripts/packages/JSONRPC/src/typed.jl:67
 [6] serve_notebook(pipename::String, outputchannel_log

In [ ]:
evd_all(20, "bursting_neuron")